In [1]:
import edgar
import pandas as pd
import re
import numpy as np

In [2]:
def ten_k_files():
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":" 0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label'])
    for comp in company_dict:
        year = 2019
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        for doc in docs:
            #print(comp)
            year -= 1
            
            docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': year,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
    return docs_df

In [22]:
def ten_k_files():
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":" 0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label', 
                                    'Filings', 'date', '1_Day', '1_Month', '3_Month', 'close',
       'close_1_Day', 'close_1_Month', 'close_3_Month'])
    for comp in company_dict:
        print(comp)
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=5)
        
        df_comp_dates = pd.DataFrame()
        
        file = f'../data/{comp}.csv'
        df_comp = pd.read_csv(file, encoding="ISO-8859-1", index_col=[0])
        df_comp.drop(['company', 'open', 'high', 'low','adjusted_close', 'volume', 'dividend_amount', 'split_coefficient' ,'crude_price'], axis=1, inplace=True)
        df_comp['date'] = pd.to_datetime(df_comp['date'])

        df_comp = df_comp.set_index('date')

        df_comp = df_comp.resample('D').mean()
        df_comp = df_comp.fillna(method='ffill')

        df_comp = df_comp.reset_index(drop=False)

        df_comp_dates = pd.read_excel("../data/dates/Document_Filing_Dates.xlsx", sheet_name=comp)
        df_comp_dates.rename(columns = {'Filing Date':'date'}, inplace = True)


        df_comp_dates['1_Day'] = df_comp_dates['date'] + np.timedelta64(1,'D')
        df_comp_dates['1_Month'] = df_comp_dates['date'] + np.timedelta64(30,'D')
        df_comp_dates['3_Month'] = df_comp_dates['date'] + np.timedelta64(90,'D')

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='date' ,how='left')

        df_comp.rename(columns = {'date':'1_Day'}, inplace = True)
        df_comp.rename(columns = {'close':'close_1_Day'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Day' ,how='left')

        df_comp.rename(columns = {'1_Day':'1_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Day':'close_1_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Month' ,how='left')

        df_comp.rename(columns = {'1_Month':'3_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Month':'close_3_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='3_Month' ,how='left')
        
                    
        df_comp_dates[df_comp_dates['Filings'] == "10-K"]
                    
                    
        df_comp_dates.reset_index(drop=True)
        df_comp_dates = df_comp_dates.reset_index(drop=True).sort_values('date', ascending=False)
        
        comp_2018 = ['RSPP','NFX']
        comp_2019 = ['FANG','CXO','MDU','PE','MRO','LPI','ECA','GPOR','RRC','EQT',"SM"]
        count = 0
        if comp in comp_2018:
            df_comp_dates_2018 = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-01-01'))].reset_index()
            start_year=2018
            #print(len(df_comp_dates_2018))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print(idx)
                
                fil = df_comp_dates_2018['Filings'][idx]
                day = df_comp_dates_2018['date'][idx]
                day_un = df_comp_dates_2018['1_Day'][idx]
                mnth = df_comp_dates_2018['1_Month'][idx]
                mnth_tre = df_comp_dates_2018['3_Month'][idx]
                close = df_comp_dates_2018['close'][idx]
                close_un = df_comp_dates_2018['close_1_Day'][idx]
                close_mnth_un = df_comp_dates_2018['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates_2018['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': start_year,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                start_year -= 1

        else:
            df_comp_dates = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-03-01'))].reset_index()
            year_start = 2019
            print(len(df_comp_dates))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print("two" + str(idx))
                #print("-----")
    
                fil = df_comp_dates['Filings'][idx]
                day = df_comp_dates['date'][idx]
                day_un = df_comp_dates['1_Day'][idx]
                mnth = df_comp_dates['1_Month'][idx]
                mnth_tre = df_comp_dates['3_Month'][idx]
                close = df_comp_dates['close'][idx]
                close_un = df_comp_dates['close_1_Day'][idx]
                close_mnth_un = df_comp_dates['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': year_start,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                year_start -= 1
        
            
        #docs_df = pd.merge(docs_df, df_comp_dates, on='Filings' ,how='left')
        #df_comp_dates
    return docs_df

In [23]:
ten_k_files()

FANG
6
count 2018:5
RSPP
count 2018:5
CXO
6
count 2018:5
MDU
6
count 2018:5
LPI
6
count 2018:5
NFX
count 2018:5
ECA
3
count 2018:3
GPOR
6
count 2018:5
RRC
6
count 2018:5
EQT
6
count 2018:5
SM
6
count 2018:5


,Filing_Type,Filing_Date,Company,Text,Label,Filings,date,1_Day,1_Month,3_Month,close,close_1_Day,close_1_Month,close_3_Month
0,10-K,2019,FANG,\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMOND...,1,10-K,2019-02-25,2019-02-26,2019-03-27,2019-05-26,105.29,105.51,102.35,NaN
1,10-K,2018,FANG,\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBAC...,1,10-K,2018-02-15,2018-02-16,2018-03-17,2018-05-16,125.15,123.88,125.51,132.13
2,10-K,2017,FANG,\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBAC...,1,10-K,2017-02-15,2017-02-16,2017-03-17,2017-05-16,111.84,109.01,104.69,103.96
3,10-K,2016,FANG,\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBAC...,1,10-K,2016-02-19,2016-02-20,2016-03-20,2016-05-19,72.08,72.08,77.62,86.93
4,10-K,2015,FANG,\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBAC...,1,10-K,2015-02-20,2015-02-21,2015-03-22,2015-05-21,75.09,75.09,72.34,80.62
5,10-K,2018,RSPP,\n10-K/A\n1\nd580186d10ka.htm\nFORM 10-K/A\n\n...,1,10-K,2018-02-27,2018-02-28,2018-03-29,2018-05-28,35.82,38.31,46.88,44.16
6,10-K,2017,RSPP,\n10-K\n1\na2017123110k.htm\nFORM 10-K\n\n\n\n...,1,10-K,2017-02-27,2017-02-28,2017-03-29,2017-05-28,40.97,39.49,41.07,37.74
7,10-K,2016,RSPP,\n10-K\n1\nrspp2016123110k.htm\n10-K\n\n\n\n\t...,1,10-K,2016-02-25,2016-02-26,2016-03-26,2016-05-25,22.80,23.02,28.57,33.09
8,10-K,2015,RSPP,\n10-K\n1\na2015123110k.htm\n10-K\n\n\n\n\t\n\...,1,10-K,2015-03-17,2015-03-18,2015-04-16,2015-06-15,27.07,25.74,28.57,30.21
9,10-K,2014,RSPP,\n10-K\n1\na2014123110k.htm\n10-K\n\n\n\n\t\n\...,1,10-K,2014-03-31,2014-04-01,2014-04-30,2014-06-29,28.89,29.20,28.35,32.30


In [233]:
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label', 'Filings', 'date', '1_Day', '1_Month', '3_Month', 'close',
       'close_1_Day', 'close_1_Month', 'close_3_Month'])
    for comp in company_dict:
        print(comp)
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=5)
        
        df_comp_dates = pd.DataFrame()
        
        file = f'../data/{comp}.csv'
        df_comp = pd.read_csv(file, encoding="ISO-8859-1", index_col=[0])
        df_comp.drop(['company', 'open', 'high', 'low','adjusted_close', 'volume', 'dividend_amount', 'split_coefficient' ,'crude_price'], axis=1, inplace=True)
        df_comp['date'] = pd.to_datetime(df_comp['date'])

        df_comp = df_comp.set_index('date')

        df_comp = df_comp.resample('D').mean()
        df_comp = df_comp.fillna(method='ffill')

        df_comp = df_comp.reset_index(drop=False)

        df_comp_dates = pd.read_excel("../data/dates/Document_Filing_Dates.xlsx", sheet_name=comp)
        df_comp_dates.rename(columns = {'Filing Date':'date'}, inplace = True)


        df_comp_dates['1_Day'] = df_comp_dates['date'] + np.timedelta64(1,'D')
        df_comp_dates['1_Month'] = df_comp_dates['date'] + np.timedelta64(30,'D')
        df_comp_dates['3_Month'] = df_comp_dates['date'] + np.timedelta64(90,'D')

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='date' ,how='left')

        df_comp.rename(columns = {'date':'1_Day'}, inplace = True)
        df_comp.rename(columns = {'close':'close_1_Day'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Day' ,how='left')

        df_comp.rename(columns = {'1_Day':'1_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Day':'close_1_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Month' ,how='left')

        df_comp.rename(columns = {'1_Month':'3_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Month':'close_3_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='3_Month' ,how='left')
        
                    
        df_comp_dates[df_comp_dates['Filings'] == "10-K"]
                    
                    
        df_comp_dates.reset_index(drop=True)
        df_comp_dates = df_comp_dates.reset_index(drop=True).sort_values('date', ascending=False)
        
        comp_2018 = ['RSPP','NFX','']
        comp_2019 = ['FANG','CXO','MDU','PE','MRO','LPI','ECA','GPOR','RRC','EQT',"SM"]
        count = 0
        if comp in comp_2018:
            df_comp_dates_2018 = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-01-01'))].reset_index()
            start_year=2018
            print(len(df_comp_dates_2018))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print(idx)
                
                fil = df_comp_dates_2018['Filings'][idx]
                day = df_comp_dates_2018['date'][idx]
                day_un = df_comp_dates_2018['1_Day'][idx]
                mnth = df_comp_dates_2018['1_Month'][idx]
                mnth_tre = df_comp_dates_2018['3_Month'][idx]
                close = df_comp_dates_2018['close'][idx]
                close_un = df_comp_dates_2018['close_1_Day'][idx]
                close_mnth_un = df_comp_dates_2018['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates_2018['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': start_year,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                start_year -= 1

        else:
            df_comp_dates = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-03-01'))].reset_index()
            year_start = 2019
            print(len(df_comp_dates_2018))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print("two" + str(idx))
                #print("-----")
    
                fil = df_comp_dates['Filings'][idx]
                day = df_comp_dates['date'][idx]
                day_un = df_comp_dates['1_Day'][idx]
                mnth = df_comp_dates['1_Month'][idx]
                mnth_tre = df_comp_dates['3_Month'][idx]
                close = df_comp_dates['close'][idx]
                close_un = df_comp_dates['close_1_Day'][idx]
                close_mnth_un = df_comp_dates['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': year_start,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                year_start -= 1
        
        
        
        #print(len(df_comp_dates))
        #print(len(docs))
        #print(df_comp_dates)
        #count = len(docs)
        
        
 
            
        #docs_df = pd.merge(docs_df, df_comp_dates, on='Filings' ,how='left')
        #df_comp_dates        
            
            

FANG
5
count 2018:5
RSPP
5
count 2018:5


In [234]:
docs_df

,Filing_Type,Filing_Date,Company,Text,Label,Filings,date,1_Day,1_Month,3_Month,close,close_1_Day,close_1_Month,close_3_Month
0,10-K,2019,FANG,\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMOND...,1,10-K,2019-02-25,2019-02-26,2019-03-27,2019-05-26,105.29,105.51,102.35,NaN
1,10-K,2018,FANG,\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBAC...,1,10-K,2018-02-15,2018-02-16,2018-03-17,2018-05-16,125.15,123.88,125.51,132.13
2,10-K,2017,FANG,\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBAC...,1,10-K,2017-02-15,2017-02-16,2017-03-17,2017-05-16,111.84,109.01,104.69,103.96
3,10-K,2016,FANG,\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBAC...,1,10-K,2016-02-19,2016-02-20,2016-03-20,2016-05-19,72.08,72.08,77.62,86.93
4,10-K,2015,FANG,\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBAC...,1,10-K,2015-02-20,2015-02-21,2015-03-22,2015-05-21,75.09,75.09,72.34,80.62
5,10-K,2018,RSPP,\n10-K/A\n1\nd580186d10ka.htm\nFORM 10-K/A\n\n...,1,10-K,2018-02-27,2018-02-28,2018-03-29,2018-05-28,35.82,38.31,46.88,44.16
6,10-K,2017,RSPP,\n10-K\n1\na2017123110k.htm\nFORM 10-K\n\n\n\n...,1,10-K,2017-02-27,2017-02-28,2017-03-29,2017-05-28,40.97,39.49,41.07,37.74
7,10-K,2016,RSPP,\n10-K\n1\nrspp2016123110k.htm\n10-K\n\n\n\n\t...,1,10-K,2016-02-25,2016-02-26,2016-03-26,2016-05-25,22.80,23.02,28.57,33.09
8,10-K,2015,RSPP,\n10-K\n1\na2015123110k.htm\n10-K\n\n\n\n\t\n\...,1,10-K,2015-03-17,2015-03-18,2015-04-16,2015-06-15,27.07,25.74,28.57,30.21
9,10-K,2014,RSPP,\n10-K\n1\na2014123110k.htm\n10-K\n\n\n\n\t\n\...,1,10-K,2014-03-31,2014-04-01,2014-04-30,2014-06-29,28.89,29.20,28.35,32.30


In [19]:


file = '../data/CXO.csv'
df_FANG = pd.read_csv(file, encoding="ISO-8859-1", index_col=[0])
df_FANG.drop(['company', 'open', 'high', 'low','adjusted_close', 'volume', 'dividend_amount', 'split_coefficient' ,'crude_price'], axis=1, inplace=True)
df_FANG['date'] = pd.to_datetime(df_FANG['date'])

df_FANG = df_FANG.set_index('date')

df_FANG = df_FANG.resample('D').mean()
df_FANG = df_FANG.fillna(method='ffill')

df_FANG = df_FANG.reset_index(drop=False)

df_FANG_dates = pd.read_excel("../data/dates/Document_Filing_Dates.xlsx", sheet_name='CXO')
df_FANG_dates.rename(columns = {'Filing Date':'date'}, inplace = True)
print(df_FANG_dates)
'''
df_FANG_dates['1_Day'] = df_FANG_dates['date'] + np.timedelta64(1,'D')
df_FANG_dates['1_Month'] = df_FANG_dates['date'] + np.timedelta64(30,'D')
df_FANG_dates['3_Month'] = df_FANG_dates['date'] + np.timedelta64(90,'D')

df_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='date' ,how='left')

df_FANG.rename(columns = {'date':'1_Day'}, inplace = True)
df_FANG.rename(columns = {'close':'close_1_Day'}, inplace = True)

df_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='1_Day' ,how='left')

df_FANG.rename(columns = {'1_Day':'1_Month'}, inplace = True)
df_FANG.rename(columns = {'close_1_Day':'close_1_Month'}, inplace = True)

df_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='1_Month' ,how='left')

df_FANG.rename(columns = {'1_Month':'3_Month'}, inplace = True)
df_FANG.rename(columns = {'close_1_Month':'close_3_Month'}, inplace = True)

df_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='3_Month' ,how='left')
'''

   Unnamed: 0           Unnamed: 1
0         NaN                  NaN
1     Filings          Filing Date
2        10-K  2014-02-21 00:00:00
3        10-Q  2014-05-12 00:00:00
4        10-Q  2014-08-07 00:00:00
5        10-Q  2014-11-06 00:00:00
6        10-K  2015-02-26 00:00:00
7        10-Q  2015-05-05 00:00:00
8        10-Q  2015-07-30 00:00:00
9        10-Q  2015-11-05 00:00:00
10       10-K  2016-02-25 00:00:00
11       10-Q  2016-05-05 00:00:00
12       10-Q  2016-08-03 00:00:00
13       10-Q  2016-11-09 00:00:00
14       10-K  2017-02-22 00:00:00
15       10-Q  2017-05-04 00:00:00
16       10-Q  2017-08-03 00:00:00
17       10-Q  2017-11-01 00:00:00
18       10-K  2018-02-21 00:00:00
19       10-Q  2018-05-02 00:00:00
20       10-Q  2018-08-02 00:00:00
21       10-Q  2018-10-31 00:00:00
22       10-K  2019-02-20 00:00:00
23       10-Q  2019-05-01 00:00:00


"\ndf_FANG_dates['1_Day'] = df_FANG_dates['date'] + np.timedelta64(1,'D')\ndf_FANG_dates['1_Month'] = df_FANG_dates['date'] + np.timedelta64(30,'D')\ndf_FANG_dates['3_Month'] = df_FANG_dates['date'] + np.timedelta64(90,'D')\n\ndf_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='date' ,how='left')\n\ndf_FANG.rename(columns = {'date':'1_Day'}, inplace = True)\ndf_FANG.rename(columns = {'close':'close_1_Day'}, inplace = True)\n\ndf_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='1_Day' ,how='left')\n\ndf_FANG.rename(columns = {'1_Day':'1_Month'}, inplace = True)\ndf_FANG.rename(columns = {'close_1_Day':'close_1_Month'}, inplace = True)\n\ndf_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='1_Month' ,how='left')\n\ndf_FANG.rename(columns = {'1_Month':'3_Month'}, inplace = True)\ndf_FANG.rename(columns = {'close_1_Month':'close_3_Month'}, inplace = True)\n\ndf_FANG_dates = pd.merge(df_FANG_dates, df_FANG, on='3_Month' ,how='left')\n"

In [176]:
#df_FANG_dates = df_FANG_dates[(df_FANG_dates['Filings'] == "10-K") & (df_FANG_dates["date"] < np.datetime64('2019-01-19'))]
df_FANG_dates

,Filings,date,1_Day,1_Month,3_Month,close,close_1_Day,close_1_Month,close_3_Month
0,10-K,2017-02-27,2017-02-28,2017-03-29,2017-05-28,11.18,11.10,11.88,10.75
1,10-Q,2017-05-04,2017-05-05,2017-06-03,2017-08-02,10.40,10.96,9.70,9.81
2,10-Q,2017-07-25,2017-07-26,2017-08-24,2017-10-23,10.35,10.02,9.29,11.15
3,10-Q,2017-11-09,2017-11-10,2017-12-09,2018-02-07,12.69,12.65,11.79,11.05
4,10-K,2018-02-26,2018-02-27,2018-03-28,2018-05-27,11.11,10.82,10.52,12.64
5,10-Q,2018-05-03,2018-05-04,2018-06-02,2018-08-01,12.74,13.16,12.40,14.02
6,10-Q,2018-08-02,2018-08-03,2018-09-01,2018-10-31,14.21,13.72,13.27,10.24
7,10-Q,2018-11-06,2018-11-07,2018-12-06,2019-02-04,8.71,8.78,6.47,7.01
8,10-K,2019-02-28,2019-03-01,2019-03-30,2019-05-29,7.25,7.26,7.24,NaN
9,10-Q,2019-05-02,2019-05-03,2019-06-01,2019-07-31,NaN,NaN,NaN,NaN


In [126]:
df_FANG_dates = df_FANG_dates[(df_FANG_dates['Filings'] == "10-K") | (df_FANG_dates['Filings'] =="10-K/A")]
#for row in df_FANG_dates:
#    print(row)
df_FANG_dates.reset_index(drop=True)
for date in df_FANG_dates['date']:
    
    if date < np.datetime64('2019-01-19'):
        #df_FANG_dates = df_FANG_dates.reset_index(drop=True).sort_values('date', ascending=False).drop([0, 1])

True
True
True
True
True


In [140]:
df_FANG_dates = df_FANG_dates[(df_FANG_dates['Filings'] == "10-K") & (df_FANG_dates["date"] < np.datetime64('2019-01-19'))]
len(df_FANG_dates)

5

In [139]:
df_FANG_dates

,Filings,date,1_Day,1_Month,3_Month,close,close_1_Day,close_1_Month,close_3_Month
0,10-K,2014-02-19,2014-02-20,2014-03-21,2014-05-20,61.04,62.67,64.22,70.16
5,10-K,2015-02-20,2015-02-21,2015-03-22,2015-05-21,75.09,75.09,72.34,80.62
10,10-K,2016-02-19,2016-02-20,2016-03-20,2016-05-19,72.08,72.08,77.62,86.93
14,10-K,2017-02-15,2017-02-16,2017-03-17,2017-05-16,111.84,109.01,104.69,103.96
18,10-K,2018-02-15,2018-02-16,2018-03-17,2018-05-16,125.15,123.88,125.51,132.13


In [3]:
def ten_q_files(company,cik):
    company = edgar.Company(company, cik)
    tree = company.getAllFilings(filingType = "10-Q")
    docs = edgar.getDocuments(tree, noOfDocuments=5)

In [2]:
company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":"0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label'])

In [11]:
    company = edgar.Company("CXO","0001358071")
    tree = company.getAllFilings(filingType = "10-k")
    docs = edgar.getDocuments(tree, noOfDocuments=5)

In [12]:
len(docs)

5

In [14]:
print(docs[0])


10-K
1
cxo201810K.htm
FORM 10-K










 






 








UNITED STATES SECURITIES AND EXCHANGE COMMISSION 

Washington,
D.C. 20549 

 

FORM 10-K 




 
  
    
  
 
    
  
 
    
  
 

  
  ☑  
  
 
    
  
 
  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES
  EXCHANGE ACT OF 1934 
  
 






For the fiscal year ended December 31, 2018

or 




 
  
    
  
 
    
  
 
    
  
 

  
   ☐   
  
 
    
  
 
  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
  SECURITIES EXCHANGE ACT OF 1934 
   
  
 






For the transition period from                     
                    
to                     
                    

 

Commission file number: 1-33615 

 

Concho Resources Inc. 

(Exact name of registrant as specified in its charter) 


 
  
    
  
 
    
  
 
    
  
 

  
  Delaware 
  
 
    
  
 
  76-0818600
  
 

  
  (State or other jurisdiction 
  
 
    
  
 
  (I.R.S. Employer 
  
 

  
  of incorporation or organization)
  
 
    
  
 
  Id

In [21]:
a = re.findall("MARCH", docs[1])

In [22]:
#print(a)
#print(a[:2])
#print(a[2:6])
if len(a) == 1:
    print(a)
else:
    print("np")

np


In [9]:
for comp in company_dict:
    print(comp)
    print(company_dict[comp])

NameError: name 'company' is not defined

In [43]:
for comp in company_dict:
        year = 2018
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-Q")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        for doc in docs:
                print(comp)
                
                
                
                qthree = re.findall("SEPTEMBER", doc)
                qtwo = re.findall("JUNE", doc)
                qone = re.findall("MARCH", doc)
                
                
                
                if len(qthree) == 0 and len(qtwo) == 0:
                        print("1", qone)
                        docs_df = docs_df.append({'Filing_Type': f'10-Q1','Filing_Date': year,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)            
                elif len(qone) == 0 and len(qtwo) == 0:
                        print("3", qthree)
                        docs_df = docs_df.append({'Filing_Type': f'10-Q3','Filing_Date': qthree,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
                else:
                    print("2", qtwo)
                    docs_df = docs_df.append({'Filing_Type': f'10-Q{qtwo}','Filing_Date': qtwo,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)

                year -= 1

FANG
1 []
FANG
3 ['SEPTEMBER']
FANG
2 ['JUNE']
FANG
1 ['MARCH']
FANG
3 ['SEPTEMBER']
RSPP
1 ['MARCH']
RSPP
3 ['SEPTEMBER']
RSPP
2 ['JUNE']
RSPP
1 ['MARCH']
RSPP
3 ['SEPTEMBER']
CXO
1 ['MARCH', 'MARCH']
CXO
3 ['SEPTEMBER']
CXO
2 ['JUNE']
CXO
1 ['MARCH']
CXO
3 ['SEPTEMBER']
MDU
1 []
MDU
1 []
MDU
1 []
MDU
1 []
MDU
1 []
LPI
1 []
LPI
1 []
LPI
1 []
LPI
1 []
LPI
1 []
NFX
1 []
NFX
1 []
NFX
1 []
NFX
1 []
NFX
1 []
ECA
1 []
ECA
1 []
ECA
1 []
ECA
1 []
ECA
1 []
GPOR
1 []
GPOR
1 []
GPOR
1 []
GPOR
1 []
GPOR
1 []
RRC
1 []
RRC
1 []
RRC
1 []
RRC
1 []
RRC
1 []
EQT
1 ['MARCH']
EQT
3 ['SEPTEMBER']
EQT
2 ['JUNE']
EQT
1 ['MARCH']
EQT
3 ['SEPTEMBER']
SM
1 []
SM
1 []
SM
1 []
SM
1 []
SM
1 []


In [ ]:
if len(qthree) == 0:
                    qthree = 3
                    print(qthree, company)
                    #docs_df = docs_df.append({'Filing_Type': f'10-Q{qthree}','Filing_Date': qthree,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
                elif len(qtwo) != 0:
                    qtwo = 2
                    print(qtwo)
                    docs_df = docs_df.append({'Filing_Type': f'10-Q{qtwo}','Filing_Date': qtwo,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
                else:
                    qone = 1
                    print(qone)
                    docs_df = docs_df.append({'Filing_Type': f'10-Q{qone}','Filing_Date': qone,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)            

In [58]:
company = edgar.Company("MDU", "0000067716")
tree = company.getFilingsUrl(filingType = "10-Q", priorTo = "20190101", noOfEntries = 5)
#docs = edgar.getDocuments(tree, noOfDocuments=5)

In [59]:
tree

'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000067716&type=10-Q&dateb=20190101&owner=include&count=5'

In [61]:
end = "/cgi-bin/browse-edgar?action=getcompany&CIK=0000067716&type=10-Q&dateb=20190101&owner=include&count=5"
docs = edgar.getDocuments(end, noOfDocuments=5)

AttributeError: 'str' object has no attribute 'xpath'

In [62]:
tree = company.getAllFilings(filingType = "10-Q", priorTo = "20190101", noOfEntries = 5)

In [66]:
docs = edgar.getDocuments(tree, noOfDocuments=5)

In [67]:
docs[0]

'\r\n  \r\n    \r\n      \r\n        \r\n          \r\n            \r\n               \r\n                Menu\r\n            \r\n            \r\n              Information\r\n              Save XBRL Instance\r\n              Save XBRL Instance\r\n              Save\r\n                  XBRL Zip File\r\n              Open as HTML\r\n              Tagged Sections\r\n              Settings\r\n              Help\r\n            \r\n          \r\n        \r\n      \r\n      \xa0\r\n      \r\n        \r\n          \r\n            \r\n              Sections\r\n            \r\n            \r\n            \r\n          \r\n        \r\n      \r\n      \xa0\r\n      \r\n        \r\n          \r\n            \r\n            \r\n            \r\n              \r\n                \r\n              \r\n            \r\n          \r\n        \r\n      \r\n      \xa0\r\n      \r\n        \r\n          \r\n            \r\n              \r\n              Data\r\n            \r\n          \r\n        \r\n   

In [153]:
ints = [8, 23, 45, 12, 78]

In [155]:
for idx, val in enumerate(ints):
    print(idx, val)

0 8
1 23
2 45
3 12
4 78


In [194]:
while i < 5:
    print(i)
    i+=1

0
1
2
3
4
